<a href="https://colab.research.google.com/github/a-nagar/vistra-intermediate/blob/main/Frequent_Pattern_Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.22.0
    Uninstalling mlxtend-0.22.0:
      Successfully uninstalled mlxtend-0.22.0


# Transactions Dataset
Let's look at a set of transactions stored in the form of a list with elements containing individual transactions.

In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [ ]:
type(dataset)

list

## Converting to Transactions Dataframe
Before we can proceed, we need to convert the transaction list using TransactionEncoder object. Notice the format of the output dataframe.

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

Apple   Corn   Dill   Eggs  Ice cream  Kidney Beans   Milk  Nutmeg  Onion  \
0  False  False  False   True      False          True   True    True   True   
1  False  False   True   True      False          True  False    True   True   
2   True  False  False   True      False          True   True   False  False   
3  False   True  False  False      False          True   True   False  False   
4  False   True  False   True       True          True  False   False   True   

   Unicorn  Yogurt  
0    False    True  
1    False    True  
2    False   False  
3     True    True  
4    False   False

# Apriori Algorithm
Let's run apriori algorithm and provide minimum support values.

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6, use_colnames=True)

support                     itemsets
0       0.8                       (Eggs)
1       1.0               (Kidney Beans)
2       0.6                       (Milk)
3       0.6                      (Onion)
4       0.6                     (Yogurt)
5       0.8         (Kidney Beans, Eggs)
6       0.6                (Onion, Eggs)
7       0.6         (Kidney Beans, Milk)
8       0.6        (Onion, Kidney Beans)
9       0.6       (Yogurt, Kidney Beans)
10      0.6  (Onion, Kidney Beans, Eggs)

# Frequent Itemsets
Let's create frequent items sets with minimum support.

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

support                     itemsets  length
0       0.8                       (Eggs)       1
1       1.0               (Kidney Beans)       1
2       0.6                       (Milk)       1
3       0.6                      (Onion)       1
4       0.6                     (Yogurt)       1
5       0.8         (Eggs, Kidney Beans)       2
6       0.6                (Eggs, Onion)       2
7       0.6         (Milk, Kidney Beans)       2
8       0.6        (Onion, Kidney Beans)       2
9       0.6       (Yogurt, Kidney Beans)       2
10      0.6  (Eggs, Onion, Kidney Beans)       3

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] >= 2) &
                   (frequent_itemsets['support'] >= 0.6) ]

support                     itemsets  length
5       0.8         (Eggs, Kidney Beans)       2
6       0.6                (Eggs, Onion)       2
7       0.6         (Milk, Kidney Beans)       2
8       0.6        (Onion, Kidney Beans)       2
9       0.6       (Yogurt, Kidney Beans)       2
10      0.6  (Eggs, Onion, Kidney Beans)       3

# Association Rules
Next, let's try to find association rules with significan confidence values from the transaction dataset.

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

antecedents            consequents  antecedent support  \
0                  (Eggs)         (Kidney Beans)                 0.8   
1          (Kidney Beans)                 (Eggs)                 1.0   
2                  (Eggs)                (Onion)                 0.8   
3                 (Onion)                 (Eggs)                 0.6   
4                  (Milk)         (Kidney Beans)                 0.6   
5                 (Onion)         (Kidney Beans)                 0.6   
6                (Yogurt)         (Kidney Beans)                 0.6   
7           (Eggs, Onion)         (Kidney Beans)                 0.6   
8    (Eggs, Kidney Beans)                (Onion)                 0.8   
9   (Onion, Kidney Beans)                 (Eggs)                 0.6   
10                 (Eggs)  (Onion, Kidney Beans)                 0.8   
11                (Onion)   (Eggs, Kidney Beans)                 0.6   

    consequent support  support  confidence  lift  leverage  conviction  
0                  1.0      0.8        1.00  1.00      0.00         inf  
1                  0.8      0.8        0.80  1.00      0.00         1.0  
2                  0.6      0.6        0.75  1.25      0.12         1.6  
3                  0.8      0.6        1.00  1.25      0.12         inf  
4                  1.0      0.6        1.00  1.00      0.00         inf  
5                  1.0      0.6        1.00  1.00      0.00         inf  
6                  1.0      0.6        1.00  1.00      0.00         inf  
7                  1.0      0.6        1.00  1.00      0.00         inf  
8                  0.6      0.6        0.75  1.25      0.12         1.6  
9                  0.8      0.6        1.00  1.25      0.12         inf  
10                 0.6      0.6        0.75  1.25      0.12         1.6  
11                 0.8      0.6        1.00  1.25      0.12         inf

Let's try to create a column with the length of items in each antecedent.

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

antecedents            consequents  antecedent support  \
0                 (Eggs)                (Onion)                 0.8   
1                (Onion)                 (Eggs)                 0.6   
2   (Eggs, Kidney Beans)                (Onion)                 0.8   
3  (Onion, Kidney Beans)                 (Eggs)                 0.6   
4                 (Eggs)  (Onion, Kidney Beans)                 0.8   
5                (Onion)   (Eggs, Kidney Beans)                 0.6   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 0.6      0.6        0.75  1.25      0.12         1.6   
1                 0.8      0.6        1.00  1.25      0.12         inf   
2                 0.6      0.6        0.75  1.25      0.12         1.6   
3                 0.8      0.6        1.00  1.25      0.12         inf   
4                 0.6      0.6        0.75  1.25      0.12         1.6   
5                 0.8      0.6        1.00  1.25      0.12         inf   

   antecedent_len  
0               1  
1               1  
2               2  
3               2  
4               1  
5               1

The above information can be used for filtering rules with sufficient number of items in antecedent or consequent.

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

antecedents consequents  antecedent support  consequent support  \
3  (Onion, Kidney Beans)      (Eggs)                 0.6                 0.8   

   support  confidence  lift  leverage  conviction  antecedent_len  
3      0.6         1.0  1.25      0.12         inf               2

# FP Growth Algorithm

FP Growth is a faster alternative to Apriori algorithm that doesn't involve explicit candidate generation.

As per documentation, "*In particular, and what makes it different from the Apriori frequent pattern mining algorithm, FP-Growth is an frequent pattern mining algorithm that does not require candidate generation. Internally, it uses a so-called FP-tree (frequent pattern tree) datastrucure without generating the candidate sets explicitely, which makes is particularly attractive for large datasets.*"

In [ ]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.6, use_colnames=True)

support                     itemsets
0       1.0               (Kidney Beans)
1       0.8                       (Eggs)
2       0.6                     (Yogurt)
3       0.6                      (Onion)
4       0.6                       (Milk)
5       0.8         (Eggs, Kidney Beans)
6       0.6       (Yogurt, Kidney Beans)
7       0.6                (Eggs, Onion)
8       0.6        (Onion, Kidney Beans)
9       0.6  (Eggs, Onion, Kidney Beans)
10      0.6         (Milk, Kidney Beans)

If you just want the maximal patterns, you can use *fpmax* algorithm.

As per documentation, "*FP-Max is a variant of FP-Growth, which focuses on obtaining maximal itemsets. An itemset X is said to maximal if X is frequent and there exists no frequent super-pattern containing X. In other words, a frequent pattern X cannot be sub-pattern of larger frequent pattern to qualify for the definition maximal itemset.*"

In [ ]:
from mlxtend.frequent_patterns import fpmax
fpmax(df, min_support=0.6, use_colnames=True)


support                     itemsets
0      0.6         (Milk, Kidney Beans)
1      0.6  (Eggs, Onion, Kidney Beans)
2      0.6       (Yogurt, Kidney Beans)

# Working With A Real Dataset
Let's work with a dataset from UCI repository: https://archive.ics.uci.edu/ml/datasets/online+retail

We will download the file and read it into a Pandas dataframe.

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
frequent_itemsets

support                                           itemsets
0   0.071429                      (4 TRADITIONAL SPINNING TOPS)
1   0.096939                       (ALARM CLOCK BAKELIKE GREEN)
2   0.102041                        (ALARM CLOCK BAKELIKE PINK)
3   0.094388                        (ALARM CLOCK BAKELIKE RED )
4   0.081633                    (BAKING SET 9 PIECE RETROSPOT )
..       ...                                                ...
85  0.084184  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...
86  0.084184  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...
87  0.102041  (SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...
88  0.099490  (SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...
89  0.081633  (SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...

[90 rows x 2 columns]

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

support                         itemsets  length
0  0.071429    (4 TRADITIONAL SPINNING TOPS)       1
1  0.096939     (ALARM CLOCK BAKELIKE GREEN)       1
2  0.102041      (ALARM CLOCK BAKELIKE PINK)       1
3  0.094388      (ALARM CLOCK BAKELIKE RED )       1
4  0.081633  (BAKING SET 9 PIECE RETROSPOT )       1

In [ ]:
pd.set_option('max_colwidth', 600)
frequent_itemsets[ (frequent_itemsets['length'] >= 2) &
                   (frequent_itemsets['support'] >= 0.07) ].sort_values(by="length", ascending=False)

support  ... length
89  0.081633  ...      4
88  0.099490  ...      3
87  0.102041  ...      3
86  0.084184  ...      3
85  0.084184  ...      3
84  0.084184  ...      3
83  0.084184  ...      3
82  0.073980  ...      3
81  0.071429  ...      3
74  0.107143  ...      2
68  0.158163  ...      2
69  0.104592  ...      2
70  0.094388  ...      2
71  0.147959  ...      2
72  0.109694  ...      2
73  0.117347  ...      2
78  0.102041  ...      2
75  0.096939  ...      2
76  0.114796  ...      2
77  0.089286  ...      2
66  0.081633  ...      2
79  0.102041  ...      2
80  0.122449  ...      2
67  0.114796  ...      2
41  0.073980  ...      2
42  0.079082  ...      2
64  0.137755  ...      2
43  0.084184  ...      2
44  0.073980  ...      2
45  0.089286  ...      2
46  0.086735  ...      2
47  0.073980  ...      2
48  0.071429  ...      2
49  0.081633  ...      2
50  0.076531  ...      2
51  0.104592  ...      2
52  0.122449  ...      2
53  0.091837  ...      2
54  0.102041  ...      2
55  0.114796  ...      2
56  0.081633  ...      2
57  0.084184  ...      2
58  0.071429  ...      2
59  0.089286  ...      2
60  0.102041  ...      2
61  0.147959  ...      2
62  0.104592  ...      2
63  0.114796  ...      2
65  0.165816  ...      2

[49 rows x 3 columns]

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

antecedents  ... conviction
0  (ALARM CLOCK BAKELIKE GREEN)  ...   3.791383
1   (ALARM CLOCK BAKELIKE PINK)  ...   3.283859
2   (ALARM CLOCK BAKELIKE RED )  ...   5.568878
3  (ALARM CLOCK BAKELIKE GREEN)  ...   4.916181
4  (ALARM CLOCK BAKELIKE GREEN)  ...   1.783673

[5 rows x 9 columns]

# Lab Assignment
You will use the MovieLens 100K dataset available from
https://grouplens.org/datasets/movielens/

We will use the version for education and research. I have already uploaded the relevant files on the server and below is the command to read the files.


In [ ]:
import pandas as pd
movies = pd.read_csv("https://an-ml.s3.us-west-1.amazonaws.com/ml-latest-small/movies.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
movies.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [ ]:
ratings = pd.read_csv("https://an-ml.s3.us-west-1.amazonaws.com/ml-latest-small/ratings.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
ratings.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Let's join the above two tables on the common key movieId.

In [ ]:
df = pd.merge(movies, ratings, on="movieId")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating   timestamp  
0       1     4.0   964982703  
1       5     4.0   847434962  
2       7     4.5  1106635946  
3      15     2.5  1510577970  
4      17     4.5  1305696483

The columns of interest to us are title and userId. Now, repeat the steps that we did earlier and find significant frequent patterns and association rules. You are free to set the selection paramters.

*Optional* - Do the results make sense? Use your knowledge of movies 😀

In [ ]:
basket = (df.loc[:10000,:]
          .groupby(['userId', 'title'])['rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('userId'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


title   Ace Ventura: Pet Detective (1994)  \
userId                                      
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   
5                                     3.0   

title   Ace Ventura: When Nature Calls (1995)  Addiction, The (1995)  \
userId                                                                 
1                                         0.0                    0.0   
2                                         0.0                    0.0   
3                                         0.0                    0.0   
4                                         0.0                    0.0   
5                                         0.0                    0.0   

title   Adventures of Priscilla, Queen of the Desert, The (1994)  \
userId                                                             
1                                                     0.0          
2                                                     0.0          
3                                                     0.0          
4                                                     4.0          
5                                                     0.0          

title   Amateur (1994)  Amazing Panda Adventure, The (1995)  \
userId                                                        
1                  0.0                                  0.0   
2                  0.0                                  0.0   
3                  0.0                                  0.0   
4                  0.0                                  0.0   
5                  0.0                                  0.0   

title   American President, The (1995)  Angels and Insects (1995)  \
userId                                                              
1                                  0.0                        0.0   
2                                  0.0                        0.0   
3                                  0.0                        0.0   
4                                  0.0                        0.0   
5                                  0.0                        0.0   

title   Anne Frank Remembered (1995)  Antonia's Line (Antonia) (1995)  ...  \
userId                                                                 ...   
1                                0.0                              0.0  ...   
2                                0.0                              0.0  ...   
3                                0.0                              0.0  ...   
4                                0.0                              0.0  ...   
5                                0.0                              0.0  ...   

title   War, The (1994)  Waterworld (1995)  \
userId                                       
1                   0.0                0.0   
2                   0.0                0.0   
3                   0.0                0.0   
4                   0.0                0.0   
5                   0.0                0.0   

title   What's Eating Gilbert Grape (1993)  When Night Is Falling (1995)  \
userId                                                                     
1                                      0.0                           0.0   
2                                      0.0                           0.0   
3                                      0.0                           0.0   
4                                      0.0                           0.0   
5                                      0.0                           0.0   

title   While You Were Sleeping (1995)  \
userId                                   
1                                  0.0   
2                                  0.0   
3                                  0.0   
4                                  0.0   
5                                  0.0   

title   White Balloon, The (Badkonake sefid) (1995)  \
userId                 

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket_sets.fillna(0, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket_sets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


title   Ace Ventura: Pet Detective (1994)  \
userId                                      
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   
5                                     1.0   
...                                   ...   
606                                   0.0   
607                                   0.0   
608                                   1.0   
609                                   0.0   
610                                   1.0   

title   Ace Ventura: When Nature Calls (1995)  Addiction, The (1995)  \
userId                                                                 
1                                           0                      0   
2                                           0                      0   
3                                           0                      0   
4                                           0                      0   
5                                           0                      0   
...                                       ...                    ...   
606                                         1                      0   
607                                         0                      0   
608                                         1                      0   
609                                         0                      0   
610                                         0                      0   

title   Adventures of Priscilla, Queen of the Desert, The (1994)  \
userId                                                             
1                                                       0          
2                                                       0          
3                                                       0          
4                                                       1          
5                                                       0          
...                                                   ...          
606                                                     1          
607                                                     0          
608                                                     0          
609                                                     0          
610                                                     0          

title   Amateur (1994)  Amazing Panda Adventure, The (1995)  \
userId                                                        
1                    0                                    0   
2                    0                                    0   
3                    0                                    0   
4                    0                                    0   
5                    0                                    0   
...                ...                                  ...   
606                  0                                    0   
607                  0                                    0   
608                  0                                    0   
609                  0                                    0   
610                  0                                    0   

title   American President, The (1995)  Angels and Insects (1995)  \
userId                                                              
1                                    0                          0   
2                                    0                          0   
3                                    0                          0   
4                                    0                          0   
5                                    0                          0   
...                                ...                        ...   
606                                  1                          0   
607                                  1                          0   
608                                  0                          0   
609          

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


support                                           itemsets
0     0.271331                (Ace Ventura: Pet Detective (1994))
1     0.150171            (Ace Ventura: When Nature Calls (1995))
2     0.119454                   (American President, The (1995))
3     0.343003                                 (Apollo 13 (1995))
4     0.218430                                      (Babe (1995))
...        ...                                                ...
4621  0.076792  (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Se...
4622  0.071672  (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), St...
4623  0.075085  (Star Wars: Episode IV - A New Hope (1977), Se...
4624  0.083618  (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), St...
4625  0.073379  (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), St...

[4626 rows x 2 columns]

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


support                                 itemsets  length
0  0.271331      (Ace Ventura: Pet Detective (1994))       1
1  0.150171  (Ace Ventura: When Nature Calls (1995))       1
2  0.119454         (American President, The (1995))       1
3  0.343003                       (Apollo 13 (1995))       1
4  0.218430                            (Babe (1995))       1

In [ ]:
pd.set_option('max_colwidth', 600)
frequent_itemsets[ (frequent_itemsets['length'] >= 2) &
                   (frequent_itemsets['support'] >= 0.07) ].sort_values(by="length", ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


support  \
4625  0.073379   
4599  0.075085   
4597  0.078498   
4596  0.071672   
4595  0.071672   
...        ...   
464   0.100683   
463   0.151877   
462   0.134812   
461   0.172355   
72    0.121160   

                                                                                                                                                                                         itemsets  \
4625  (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Star Wars: Episode IV - A New Hope (1977), Shawshank Redemption, The (1994), Pulp Fiction (1994), Usual Suspects, The (1995), Toy Story (1995))   
4599                       (Braveheart (1995), Batman Forever (1995), Seven (a.k.a. Se7en) (1995), Shawshank Redemption, The (1994), Pulp Fiction (1994), Dumb & Dumber (Dumb and Dumber) (1994))   
4597                         (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Braveheart (1995), Apollo 13 (1995), Seven (a.k.a. Se7en) (1995), Shawshank Redemption, The (1994), Pulp Fiction (1994))   
4596                                                  (Braveheart (1995), Apollo 13 (1995), Seven (a.k.a. Se7en) (1995), Shawshank Redemption, The (1994), Pulp Fiction (1994), Toy Story (1995))   
4595                                                       (Braveheart (1995), Batman Forever (1995), Waterworld (1995), Apollo 13 (1995), Shawshank Redemption, The (1994), Pulp Fiction (1994))   
...                                                                                                                                                                                           ...   
464                                                                                                                                             (Star Trek: Generations (1994), GoldenEye (1995))   
463                                                                                                                                          (GoldenEye (1995), Shawshank Redemption, The (1994))   
462                                                                                                                                               (GoldenEye (1995), Seven (a.k.a. Se7en) (1995))   
461                                                                                                                                                       (GoldenEye (1995), Pulp Fiction (1994))   
72                                                                                                                     (Ace Ventura: Pet Detective (1994), Ace Ventura: When Nature Calls (1995))   

      length  
4625       6  
4599       6  
4597       6  
4596       6  
4595       6  
...      ...  
464        2  
463        2  
462        2  
461        2  
72         2  

[4554 rows x 3 columns]

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values("confidence", ascending=False)[:100]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


antecedents  \
53444                 (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Braveheart (1995), Seven (a.k.a. Se7en) (1995), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Usual Suspects, The (1995))   
50829                                                                                         (Usual Suspects, The (1995), Taxi Driver (1976), Seven (a.k.a. Se7en) (1995), Shawshank Redemption, The (1994))   
11032                                                                                                       (Batman Forever (1995), Die Hard: With a Vengeance (1995), Ace Ventura: When Nature Calls (1995))   
53382                 (Braveheart (1995), Star Wars: Episode IV - A New Hope (1977), Seven (a.k.a. Se7en) (1995), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Usual Suspects, The (1995))   
54374  (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Seven (a.k.a. Se7en) (1995), Shawshank Redemption, The (1994), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Usual Suspects, The (1995))   
...                                                                                                                                                                                                       ...   
40270                                                                        (Ace Ventura: Pet Detective (1994), Dumb & Dumber (Dumb and Dumber) (1994), Shawshank Redemption, The (1994), Braveheart (1995))   
28727                                                                                         (Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Usual Suspects, The (1995), Braveheart (1995))   
41350                                                              (Ace Ventura: Pet Detective (1994), Dumb & Dumber (Dumb and Dumber) (1994), Seven (a.k.a. Se7en) (1995), Shawshank Redemption, The (1994))   
47529                                                      (Shawshank Redemption, The (1994), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Seven (a.k.a. Se7en) (1995), Braveheart (1995))   
9976                                                                                                                                 (Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Natural Born Killers (1994))   

                               consequents  antecedent support  \
53444                (Pulp Fiction (1994))            0.071672   
50829                (Pulp Fiction (1994))            0.078498   
11032  (Ace Ventura: Pet Detective (1994))            0.075085   
53382                (Pulp Fiction (1994))            0.075085   
54374                (Pulp Fiction (1994))            0.076792   
...                                    ...                 ...   
40270                (Pulp Fiction (1994))            0.100683   
28727                (Pulp Fiction (1994))            0.098976   
41350                (Pulp Fiction (1994))            0.098976   
47529                (Pulp Fiction (1994))            0.098976   
9976                 (Pulp Fiction (1994))            0.097270   

       consequent support   support  confidence      lift  leverage  \
53444            0.520478  0.071672    1.000000  1.921311  0.034368   
50829            0.520478  0.078498    1.000000  1.921311  0.037642   
11032            0.271331  0.075085    1.000000  3.685535  0.054712   
53382            0.520478  0.075085    1.000000  1.921311  0.036005   
54374            0.520478  0.076792    1.000000  1.921311  0.036823   
...                   ...       ...         ...       ...       ...   
40270            0.520478  0.097270    0.966102  1.856182  0.044867   
28727            0.520478  0.095563    0.965517  1.855059  0.044048   
41350            0.520478  0.095563    0.965517  1.855059  0.044048   
47529            0.520478  0.095563    0.965517  1.855059  0.044048   
9976             0.520478  0.093857    0.964912  1.853897  0.043230   

       conviction  zhangs_metric  
5344